### Exercise time

In [115]:
!wget -O Chinook_Sqlite.sqlite https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite?raw=true

--2020-11-27 19:04:25--  https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite?raw=true
Resolving github.com (github.com)... 13.234.176.102
Connecting to github.com (github.com)|13.234.176.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite [following]
--2020-11-27 19:04:26--  https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite [following]
--2020-11-27 19:04:26--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving raw.githubusercontent.com (raw.githubuserco

In [3]:
!ls

 1.Python_Basics_and_Strings.ipynb	      6.Advanced_Topics_in_Python.ipynb
 2.Python_Lists_Tuples.ipynb		      Chinook_Sqlite.sqlite
 3.Python_Sets_Dictionaries_and_Files.ipynb  'Copy of SQLBasics.ipynb'
 4.Python_Regex_Utils_Exceptions.ipynb	      sample.db
 5.Introduction_to_OOPs_Concepts.ipynb	      sample_text_file


In [260]:
import pandas as pd
pd.options.display.max_colwidth=999

import sqlite3

conn = sqlite3.connect('Chinook_Sqlite.sqlite')

In [261]:
# Let us setup some helper functions
cur = conn.cursor()

def execute(queryString):
  conn.execute(queryString)
  conn.commit()
  
  
def executemany(queryString,data):
  cur.executemany(queryString,data)
  conn.commit()

def read_sql(sql_query):
  return pd.read_sql(sql_query,conn)

def list_tables():
  return read_sql("SELECT * FROM sqlite_master WHERE type='table' and name not like 'sqlite_%' ORDER BY name;")


In [262]:
list_tables()

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,2,"CREATE TABLE [Album]\n(\n [AlbumId] INTEGER NOT NULL,\n [Title] NVARCHAR(160) NOT NULL,\n [ArtistId] INTEGER NOT NULL,\n CONSTRAINT [PK_Album] PRIMARY KEY ([AlbumId]),\n FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"
1,table,Artist,Artist,3,"CREATE TABLE [Artist]\n(\n [ArtistId] INTEGER NOT NULL,\n [Name] NVARCHAR(120),\n CONSTRAINT [PK_Artist] PRIMARY KEY ([ArtistId])\n)"
2,table,Customer,Customer,4,"CREATE TABLE [Customer]\n(\n [CustomerId] INTEGER NOT NULL,\n [FirstName] NVARCHAR(40) NOT NULL,\n [LastName] NVARCHAR(20) NOT NULL,\n [Company] NVARCHAR(80),\n [Address] NVARCHAR(70),\n [City] NVARCHAR(40),\n [State] NVARCHAR(40),\n [Country] NVARCHAR(40),\n [PostalCode] NVARCHAR(10),\n [Phone] NVARCHAR(24),\n [Fax] NVARCHAR(24),\n [Email] NVARCHAR(60) NOT NULL,\n [SupportRepId] INTEGER,\n CONSTRAINT [PK_Customer] PRIMARY KEY ([CustomerId]),\n FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"
3,table,Employee,Employee,7,"CREATE TABLE [Employee]\n(\n [EmployeeId] INTEGER NOT NULL,\n [LastName] NVARCHAR(20) NOT NULL,\n [FirstName] NVARCHAR(20) NOT NULL,\n [Title] NVARCHAR(30),\n [ReportsTo] INTEGER,\n [BirthDate] DATETIME,\n [HireDate] DATETIME,\n [Address] NVARCHAR(70),\n [City] NVARCHAR(40),\n [State] NVARCHAR(40),\n [Country] NVARCHAR(40),\n [PostalCode] NVARCHAR(10),\n [Phone] NVARCHAR(24),\n [Fax] NVARCHAR(24),\n [Email] NVARCHAR(60),\n CONSTRAINT [PK_Employee] PRIMARY KEY ([EmployeeId]),\n FOREIGN KEY ([ReportsTo]) REFERENCES [Employee] ([EmployeeId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"
4,table,Genre,Genre,9,"CREATE TABLE [Genre]\n(\n [GenreId] INTEGER NOT NULL,\n [Name] NVARCHAR(120),\n CONSTRAINT [PK_Genre] PRIMARY KEY ([GenreId])\n)"
5,table,Invoice,Invoice,10,"CREATE TABLE [Invoice]\n(\n [InvoiceId] INTEGER NOT NULL,\n [CustomerId] INTEGER NOT NULL,\n [InvoiceDate] DATETIME NOT NULL,\n [BillingAddress] NVARCHAR(70),\n [BillingCity] NVARCHAR(40),\n [BillingState] NVARCHAR(40),\n [BillingCountry] NVARCHAR(40),\n [BillingPostalCode] NVARCHAR(10),\n [Total] NUMERIC(10,2) NOT NULL,\n CONSTRAINT [PK_Invoice] PRIMARY KEY ([InvoiceId]),\n FOREIGN KEY ([CustomerId]) REFERENCES [Customer] ([CustomerId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"
6,table,InvoiceLine,InvoiceLine,12,"CREATE TABLE [InvoiceLine]\n(\n [InvoiceLineId] INTEGER NOT NULL,\n [InvoiceId] INTEGER NOT NULL,\n [TrackId] INTEGER NOT NULL,\n [UnitPrice] NUMERIC(10,2) NOT NULL,\n [Quantity] INTEGER NOT NULL,\n CONSTRAINT [PK_InvoiceLine] PRIMARY KEY ([InvoiceLineId]),\n FOREIGN KEY ([InvoiceId]) REFERENCES [Invoice] ([InvoiceId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"
7,table,MediaType,MediaType,14,"CREATE TABLE [MediaType]\n(\n [MediaTypeId] INTEGER NOT NULL,\n [Name] NVARCHAR(120),\n CONSTRAINT [PK_MediaType] PRIMARY KEY ([MediaTypeId])\n)"
8,table,Playlist,Playlist,15,"CREATE TABLE [Playlist]\n(\n [PlaylistId] INTEGER NOT NULL,\n [Name] NVARCHAR(120),\n CONSTRAINT [PK_Playlist] PRIMARY KEY ([PlaylistId])\n)"
9,table,PlaylistTrack,PlaylistTrack,16,"CREATE TABLE [PlaylistTrack]\n(\n [PlaylistId] INTEGER NOT NULL,\n [TrackId] INTEGER NOT NULL,\n CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY ([PlaylistId], [TrackId]),\n FOREIGN KEY ([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)"


### Some questions you can find answer for 

1. Find the album that has maximum number of tracks
2. Which is the album that contains the longest track
3. What is the avergage track duration for each genre
4. What is the most frequent media type
5. Who is the customer who has bought maximum number of tracks
6. Find the playlists that have a total duration less than a hour
7. Find the playlists that have maximum number of tracks

### Answer - nov-30-2020

In [265]:
#Find the album that has maximum number of tracks
query = '''
select al.AlbumId , al.Title , Count(*) Max_Notracks
from Album al join track tk on (al.AlbumId = tk.AlbumId) group by al.AlbumId , al.Title
order by Max_Notracks DESC limit 1
'''

read_sql(query)

,AlbumId,Title,Max_Notracks
0,141,Greatest Hits,57


In [266]:
#Which is the album that contains the longest track
query = '''
select 
t.AlbumId,t.Name,a.title,t.Milliseconds 
FROM Track t join album a on (t.albumid = a.albumid)
WHERE Milliseconds in 
(SELECT max(Milliseconds) from Track t)
'''


read_sql(query)

,AlbumId,Name,Title,Milliseconds
0,227,Occupation / Precipice,"Battlestar Galactica, Season 3",5286953


In [267]:
#What is the avergage track duration for each genre
query = '''
select g.genreId,t.Name,avg(t.Milliseconds)/1000/60.0 as Average_minutes
FROM Track t join genre g on (t.trackid = g.genreid)
group by g.genreid, t.Name

'''
read_sql(query)

,GenreId,Name,Average_minutes
0,1,For Those About To Rock (We Salute You),5.728650
1,2,Balls to the Wall,5.709367
2,3,Fast As a Shark,3.843650
3,4,Restless and Wild,4.200850
4,5,Princess of the Dawn,6.256967
5,6,Put The Finger On You,3.427700
6,7,Let's Get It Up,3.898767
7,8,Inject The Venom,3.513900
8,9,Snowballed,3.385033
9,10,Evil Walks,4.391617


In [120]:
read_sql('select * from mediatype ')

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [121]:
#What is the most frequent media type
query = """
SELECT a.MediaTypeId,a.Name,count(*) as frequent
FROM
MediaType a join Track b on (a.MediaTypeId = b.MediaTypeId)
GROUP BY a.MediaTypeId,a.Name

"""
read_sql(query)

,MediaTypeId,Name,frequent
0,1,MPEG audio file,3034
1,2,Protected AAC audio file,237
2,3,Protected MPEG-4 video file,214
3,4,Purchased AAC audio file,7
4,5,AAC audio file,11


In [129]:
read_sql('select Customerid,firstname,city,state,country,phone from customer limit 5')

,CustomerId,FirstName,City,State,Country,Phone
0,1,Luís,São José dos Campos,SP,Brazil,+55 (12) 3923-5555
1,2,Leonie,Stuttgart,None,Germany,+49 0711 2842222
2,3,François,Montréal,QC,Canada,+1 (514) 721-4711
3,4,Bjørn,Oslo,None,Norway,+47 22 44 22 22
4,5,František,Prague,None,Czech Republic,+420 2 4172 5555


In [176]:
#Who is the customer who has bought maximum number of tracks
query = '''
select c.customerid,c.Find the playlists that have a total duration less than a hourfirstname,sum(il.quantity) as bought_maxtracks from invoiceline il 
join invoice i on (i.invoiceid = il.invoiceid) join customer c on (i.customerid = c.customerid)
group by c.customerid,c.firstname
'''
quer = '''
SELECT c.CustomerId,c.FirstName,sum(il.Quantity)
FROM
Customer c join Invoice i on (i.CustomerId = c.CustomerId)
join InvoiceLine il on (i.InvoiceId = il.InvoiceId)
GROUP BY c.CustomerId,c.FirstName '''
read_sql(query)

,CustomerId,FirstName,bought_maxtracks
0,1,Luís,38
1,2,Leonie,38
2,3,François,38
3,4,Bjørn,38
4,5,František,38
5,6,Helena,38
6,7,Astrid,38
7,8,Daan,38
8,9,Kara,38
9,10,Eduardo,38


In [228]:
#Find the playlists that have a total duration less than a hour
query = '''
select p.playlistid,p.name , sum(t.milliseconds/(1000.0*60*60)) as Hours from playlisttrack pt join playlist p on (pt.playlistid = p.playlistid) 
join track t on (t.trackid = pt.trackid) group by p.playlistid , p.name having Hours < 1.0
'''
read_sql(query)

,PlaylistId,Name,Hours
0,9,Music Videos,0.081748
1,18,On-The-Go 1,0.054850


In [257]:
#Find the playlists that have maximum number of tracks
query = '''
select p.name,count(t.trackid) as Max_Notracks  from track t 
join playlisttrack plt on (t.trackid = plt.trackid)
join playlist p on (plt .playlistid = p.playlistid)
group by p.Name, p.playlistid

'''
read_sql(query)


,Name,Max_Notracks
0,90’s Music,1477
1,Brazilian Music,39
2,Classical,75
3,Classical 101 - Deep Cuts,25
4,Classical 101 - Next Steps,25
5,Classical 101 - The Basics,25
6,Grunge,15
7,Heavy Metal Classic,26
8,Music,3290
9,Music,3290


### -----------------------------------------------------------------------------------------------


### Column info

In [15]:
read_sql("pragma table_info('Album')")

,cid,name,type,notnull,dflt_value,pk
0,0,AlbumId,INTEGER,1,None,1
1,1,Title,NVARCHAR(160),1,None,0
2,2,ArtistId,INTEGER,1,None,0


### Create VIEW

In [195]:
query = """
CREATE VIEW TrackDuration as
SELECT 
a.Title as AlbumTitle,t.Name as TrackName, (MilliSeconds/1000/60.0) as Minutes
FROM
Track t join Album a on (a.AlbumId = t.AlbumId)
"""
execute(query)

In [18]:
read_sql("select * from TrackDuration limit 5")

,AlbumTitle,TrackName,Minutes
0,For Those About To Rock We Salute You,For Those About To Rock (We Salute You),5.716667
1,Balls to the Wall,Balls to the Wall,5.700000
2,Restless and Wild,Fast As a Shark,3.833333
3,Restless and Wild,Restless and Wild,4.200000
4,Restless and Wild,Princess of the Dawn,6.250000
